In [3]:
import json
import os

import tqdm

import src.visualization_utils as vu

required_paths = ["/ai4eutils", "/CameraTraps", "/yolov5"]
python_path = os.environ.get("PYTHONPATH", "")
root_path = os.getcwd()

for path in required_paths:
    if not any(p.endswith(path) for p in python_path.split(":")):
        python_path += f":{root_path}/data/external{path}"

os.environ["PYTHONPATH"] = python_path

In [4]:
DATASET_PATH = os.path.abspath("./data/interim/emptyNonEmptyDataset")
MASKED_PATH = os.path.abspath("./resources/masked_images")
DETECTION_JSON = os.path.abspath("./resources/json/search_0_003.json")

THRESHOLD_NAME = (
    os.path.basename(DETECTION_JSON).replace(os.path.basename(DETECTION_JSON).split("_")[0] + "_", "")).replace(".json",
                                                                                                                "")

MASKED_PATH = os.path.join(MASKED_PATH, THRESHOLD_NAME)

In [5]:
with open(DETECTION_JSON, "r") as f:
    result = json.load(f)

for image in tqdm.tqdm(result["images"]):
    input_file = os.path.join(DATASET_PATH, image["file"])
    output_file = os.path.join(MASKED_PATH, image["file"])
    detections = image["detections"]

    vu.apply_mask_on_file(input_file, output_file, detections)


100%|██████████| 28570/28570 [04:48<00:00, 99.07it/s] 
